orden de arranque --> ubun3, ubun1, ubun2

hay que levantar los servidores --> archivos .conf y slv.conf

In [2]:
import redis

r1 = redis.Redis(host='127.0.0.1', port=7000)
r2 = redis.Redis(host='127.0.0.1', port=7002)
r3 = redis.Redis(host='127.0.0.1', port=7004)

In [3]:
r1.set('admin', 'jaime')

True

In [4]:
r1.get('admin')

b'jaime'

In [19]:
try:
    r1.set('foo', 'bar')
    print('Nodo 1 exitoso')
except:
    try:
        r2.set('foo', 'bar')
        print('Nodo 2 exitoso')
    except:
        try:
            r3.set('foo','bar')
            print('Nodo 3 exitoso')
        except:
            print('No se puede completar la acción')

Nodo 3 exitoso


REDISCLUSTER NO HA FUNCIONADO --> NO LEE LA LIBRERIA --> UTILIZAMOS CRMOD

In [ ]:
from rediscluster import RedisCluster

startup_nodes = [{"host": "127.0.0.1", "port": "7000"}]
rc = RedisCluster(startup_nodes=startup_nodes, decode_responses=True)

rc.cluster_keyslot("foo")

In [7]:
import crcmod
def getSlot(key):
    crc16 = crcmod.mkCrcFun(0x11021, rev=False, initCrc=0x0000, xorOut=0x0000)
    return crc16(key) % 16384

In [8]:
getSlot(b'foo')

12182

In [5]:
SlotToServer = dict([(i, r1) for i in range(5461)] + [(j, r2) for j in range(5461, 10923)] + [(k, r3) for k in range(10923, 16384)])

In [9]:
r = SlotToServer[getSlot(b'foo')]
r.set('foo', 'bar')

True

Apartado 1.5

SET / GET

In [10]:
r = SlotToServer[getSlot(b'foo')]
r.set('foo', 'bar')

True

In [11]:
r.get('foo')

b'bar'

INCR

In [17]:
r = SlotToServer[getSlot(b'logincount')]

In [18]:
r.set('logincount', 1)

True

In [19]:
r.incr('logincount')

2

In [20]:
r.get('logincount')

b'2'

LISTAS

In [21]:
r = SlotToServer[getSlot(b'users')]

In [23]:
r.lpush('users', 'steve', 'bob')

2

In [24]:
r.lindex('users', 1)

b'steve'

In [27]:
r.lrange('users', 0 , -1)

[b'bob', b'steve']

SETS

In [28]:
r = SlotToServer[getSlot(b'fruits')]

In [29]:
r.sadd('fruits', 'apples', 'oranges')

2

In [30]:
r.smembers('fruits')

{b'apples', b'oranges'}

In [31]:
r.sismember('fruits', 'oranges')

True

In [32]:
r.sismember('fruits', 'melon')

False

HASHES

In [35]:
r = SlotToServer[getSlot(b'team')]

In [37]:
r.hset('team', 'numPlayers', 20)

1

In [38]:
r.hset('team', 'trainers', 4)

1

In [40]:
r.hget('team', 'numPlayers')

b'20'

In [39]:
r.hget('team', 'trainers')

b'4'

SORTED SET

In [68]:
r = SlotToServer[getSlot(b'RMFans')]

In [70]:
r.zadd('RMFans', {'javier':50124,'kylian':2})

0

In [72]:
r.zrange('RMFans', 0, 1, withscores=True)

[(b'kylian', 2.0), (b'javier', 50124.0)]

PUBLISH

In [81]:
r = SlotToServer[getSlot(b'channel')]

In [ ]:
p = r.pubsub()
p.subscribe('ondacero:103.7')

In [ ]:
p = r.pubsub()
p.subscribe('cope:99.9')

In [98]:
r.publish('channel', 'cope:99.9')

0

In [96]:
r.publish('channel', 'ondacero:103.7')

0

In [97]:
p = r.pubsub()
p.subscribe('ondacero:103.7')

GEO

In [85]:
r = SlotToServer[getSlot(b'ubicaciones')]

In [87]:
r.geoadd('ubicaciones', 40.471028, -3.807628, 'casaJaime')

1

In [88]:
r.geoadd('ubicaciones', 40.4950654269385, -3.714126648731461, 'casaJavi')

1

In [89]:
r.geodist('ubicaciones', 'casaJaime', 'casaJavi')

10736.772